In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
#from keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
%matplotlib inline
import gensim.models.keyedvectors as word2vec
import pandas as pd
import gc
import gensim
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import GRU
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score

In [3]:
from google.colab import drive
#This command mounts the drive in /content/drive location.
#On execution it produces a link and subsequently an authentication code
drive.mount('/content/drive')

Mounted at /content/drive


Loading training data and test data

In [4]:
from google.colab import files
import pandas as pd
import io
#uploaded = files.upload()
dataset = pd.read_csv("/content/drive/My Drive/Hadoop/mini_project1/train.csv")
test = pd.read_csv("/content/drive/My Drive/Hadoop/mini_project1/test.csv")

Loading testing labels data

In [5]:
test_labelss = pd.read_csv("/content/drive/My Drive/Hadoop/mini_project1/test_labels.csv")
test_labelss.drop('id',axis=1,inplace=True)
print(test_labelss)

        toxic  severe_toxic  obscene  threat  insult  identity_hate
0          -1            -1       -1      -1      -1             -1
1          -1            -1       -1      -1      -1             -1
2          -1            -1       -1      -1      -1             -1
3          -1            -1       -1      -1      -1             -1
4          -1            -1       -1      -1      -1             -1
...       ...           ...      ...     ...     ...            ...
153159     -1            -1       -1      -1      -1             -1
153160     -1            -1       -1      -1      -1             -1
153161     -1            -1       -1      -1      -1             -1
153162     -1            -1       -1      -1      -1             -1
153163     -1            -1       -1      -1      -1             -1

[153164 rows x 6 columns]


segregating comments and labels

In [6]:
sentiment = dataset['comment_text'].values
y_list = ["toxic","severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = dataset[y_list].values
print(sentiment)
print(y)

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead o

Tokenizing the words with maximum number of words taken into consideration = 20000

In [7]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(sentiment))

Padding the data

In [8]:
from keras.preprocessing import text, sequence

seq = tokenizer.texts_to_sequences(sentiment)
pad = sequence.pad_sequences(seq, maxlen=700)

Tokenizing and padding the test data

In [9]:
test = test['comment_text'].values
test_seq = tokenizer.texts_to_sequences(test)
test_pad = sequence.pad_sequences(test_seq, maxlen=700)

This is Bidirectional LSTM, this model is overfitting using 10 epochs. There is another model with lesser epochs in another IPYNB file attached with the assignment

In [10]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers


def model_add_lstm(embedding_dim,hidden_layer):
    inputs = Input(shape=(700, ))
    x = Embedding(20000, embedding_dim)(inputs)
    x = Bidirectional(LSTM(hidden_layer))(x)
    #x = Dropout(0.1)(x)
    #x = Dense(hidden_layer, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    #model = model_add(embedding_dim,hidden_layer)
    #print(model.summary())
    return model


This model uses embedding dimensions = 10, length of sequence = 700, hidden layer = 128

In [ ]:
embedding_dim,hidden_layer = 10,128
model = model_add_lstm(embedding_dim,hidden_layer)
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 700)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 700, 10)           200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              142336    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 343,878
Trainable params: 343,878
Non-trainable params: 0
_____________________________________________________

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
res = model.fit(pad, y, batch_size=10, epochs=10, validation_split=0.1, callbacks=early)

Epoch 1/10
14362/14362 [==============================] - 733s 51ms/step - loss: 0.0476 - accuracy: 0.9941 - val_loss: 0.0494 - val_accuracy: 0.9940
Epoch 2/10
14362/14362 [==============================] - 725s 51ms/step - loss: 0.0438 - accuracy: 0.9938 - val_loss: 0.0481 - val_accuracy: 0.9940
Epoch 3/10
14362/14362 [==============================] - 724s 50ms/step - loss: 0.0416 - accuracy: 0.9906 - val_loss: 0.0474 - val_accuracy: 0.9938
Epoch 4/10
14362/14362 [==============================] - 724s 50ms/step - loss: 0.0394 - accuracy: 0.9925 - val_loss: 0.0466 - val_accuracy: 0.9938
Epoch 5/10
14362/14362 [==============================] - 725s 51ms/step - loss: 0.0375 - accuracy: 0.9922 - val_loss: 0.0482 - val_accuracy: 0.9937
Epoch 6/10
14362/14362 [==============================] - 725s 50ms/step - loss: 0.0356 - accuracy: 0.9921 - val_loss: 0.0486 - val_accuracy: 0.9932
Epoch 7/10
14362/14362 [==============================] - 725s 50ms/step - loss: 0.0336 - accuracy: 0.9885

In [ ]:
pad.shape

In [ ]:
y_pred_lstm = model.predict(test_pad)

In [ ]:
y_list = ["toxic","severe_toxic", "obscene", "threat", "insult", "identity_hate"]
ref_labels = test_labelss[y_list].values

In [ ]:
scores_lstm = model.evaluate(test_pad,ref_labels, verbose=0)
print('Test loss:', scores_lstm[0])
print('Test accuracy:', scores_lstm[1])

Test loss: -5.333376884460449
Test accuracy: 0.9865503907203674


Bidirectional Multiple LSTM

In [ ]:
def model_add_multi(embedding_dim,hidden_layer):
    inputs = Input(shape=(700, ))
    x = Embedding(20000, embedding_dim)(inputs)
    x = Bidirectional(LSTM(hidden_layer, return_sequences = True))(x)
    x = Bidirectional(LSTM(hidden_layer))(x)
    x = Dropout(0.1)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    #model = model_add(embedding_dim,hidden_layer)
    #print(model.summary())
    return model

In [17]:
embedding_dim,hidden_layer = 10,64
model_multi = model_add_multi(embedding_dim,hidden_layer)
print(model_multi.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 700)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 700, 10)           200000    
                                                                 
 bidirectional_4 (Bidirectio  (None, 700, 128)         38400     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 774 

This model uses maximum length = 700, embedding dimension = 10, first hidden layer = 64, second hidden layer = 64

In [18]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)
res = model_multi.fit(pad, y, batch_size=64, epochs=6, validation_split=0.1, callbacks=early)

Epoch 1/6
2244/2244 [==============================] - 834s 362ms/step - loss: 0.0825 - accuracy: 0.9523 - val_loss: 0.0543 - val_accuracy: 0.9940
Epoch 2/6
2244/2244 [==============================] - 812s 362ms/step - loss: 0.0485 - accuracy: 0.9941 - val_loss: 0.0496 - val_accuracy: 0.9939
Epoch 3/6
2244/2244 [==============================] - 809s 361ms/step - loss: 0.0450 - accuracy: 0.9941 - val_loss: 0.0499 - val_accuracy: 0.9940
Epoch 4/6
2244/2244 [==============================] - 809s 361ms/step - loss: 0.0425 - accuracy: 0.9940 - val_loss: 0.0498 - val_accuracy: 0.9937


In [19]:
y_pred_lstm_multi = model_multi.predict(test_pad)

In [20]:
y_list = ["toxic","severe_toxic", "obscene", "threat", "insult", "identity_hate"]
ref_labels = test_labelss[y_list].values
scores_lstm_multi = model_multi.evaluate(test_pad,ref_labels, verbose=0)
print('Test loss:', scores_lstm_multi[0])
print('Test accuracy:', scores_lstm_multi[1])

Test loss: -3.8450260162353516
Test accuracy: 0.9969574809074402
